# Demonstration of PyCIRAS

## Usage

The user needs to import the setup script to build the environment

In [1]:
import setup_notebook_environment

After this - the user can access the functionality through a variety of entry points on the form **pyciras.run_**

In [2]:
import pyciras

### Full Analysis - Code Quality, Git metrics, Unit-Testing, Stargazers data

The user can specify a list of git repos they want to analyze directly in the notebook, or in the default **/repos.txt** file.

In [3]:
repos = ['https://github.com/SamuelThand/TDD-Hangman',
         'https://github.com/coinse/sadl',
         'https://github.com/zhangj111/astnn']

There are some options for the analysis:

- Repo URLs
- Chunk Size - How many repos the program should analyze before writing results to disk
- Paralellism - If the program should use subprocesses equal to the chunk size, to speed up computation
- Remove Repos - The program can remove the downloaded repositories after the analysis, to save storage space

For long-running analysis, we support **ntfy** push notifications, so you can leave it running and get notified when completed.

In [5]:
pyciras.run_full_analysis(repo_urls=repos, chunk_size = 2, parallelism = False, remove_repos_after_completion=False)

Output()

Output()

Output()

Output()

Output()

...

Output()

Output()

Output()

Output()

Output()

### Single Analysis - More targeted and efficient mining for specific data

In [ ]:
pyciras.run_repo_cloner()

In [ ]:
pyciras.run_pydriller_analysis()

In [ ]:
pyciras.run_code_quality_analysis()

In [ ]:
pyciras.run_unit_testing_analysis()

In [ ]:
pyciras.run_stargazers_analysis()